In [1]:
import numpy as np
import pandas as pd
import re
import time
import matplotlib.pyplot as plt

from konlpy.tag import Okt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, Dense, LSTM

import reservoirpy as rpy
from reservoirpy.nodes import Reservoir, Ridge, Input, FORCE

from sklearn.metrics import accuracy_score

In [2]:
DATA_TRAIN_PATH = tf.keras.utils.get_file("ratings_train.txt",
                        "https://raw.github.com/ironmanciti/Infran_NLP/main/data/naver_movie/ratings_train.txt")
DATA_TEST_PATH = tf.keras.utils.get_file("ratings_test.txt",
                        "https://raw.github.com/ironmanciti/Infran_NLP/main/data/naver_movie/ratings_test.txt")

In [3]:
train_data = pd.read_csv(DATA_TRAIN_PATH, delimiter='\t')
test_data = pd.read_csv(DATA_TEST_PATH, delimiter='\t')

In [4]:
# 훈련 데이터에서 50,000개의 샘플을 무작위로 선택 (재현성을 위해 random_state=1 사용)
train_data = train_data.sample(n=50000, random_state=1)

# 테스트 데이터에서 5,000개의 샘플을 무작위로 선택 (재현성을 위해 random_state=1 사용)
test_data = test_data.sample(n=5000, random_state=1)

print(train_data.shape)
print(test_data.shape)

(50000, 3)
(5000, 3)


In [5]:
train_data.dropna(inplace=True)

test_data.dropna(inplace=True)

In [6]:
def preprocessing(sentence, remove_stopwords=True):
    okt = Okt()
    # 불용어 리스트 정의 (현재는 빈 리스트로 설정)
    # stop_words = set(['에', '은', '는', '이', '가', '그리고', '것', '들', '수', '등', '로', '을', '를', '만', '도', '아', '의', '그', '다'])
    stop_words = []

    # 개행문자 제거
    sentence = re.sub('\\\\n', ' ', sentence)

    # 한글 외의 모든 문자 제거
    sentence = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ ]', '', sentence)

    # 형태소 분석 및 어간 추출
    sentence = okt.morphs(sentence, stem=True)

    # 불용어 제거 옵션이 True인 경우, 불용어 리스트에 포함되지 않은 토큰만 선택
    if remove_stopwords:
        sentence = [token for token in sentence if not token in stop_words]

    # 전처리된 문장 반환
    return sentence

In [7]:
# 훈련 문장과 레이블을 저장할 리스트 초기화
train_sentences = []
train_labels = []

# 테스트 문장과 레이블을 저장할 리스트 초기화
test_sentences = []
test_labels = []

# 시작 시간 기록
start = time.time()

# 훈련 데이터 전처리
for i, (sent, label) in enumerate(zip(train_data['document'], train_data['label'])):
    # 10,000개마다 진행 상황 출력
    if i % 10000 == 0:
        print(f"Train processed = {i}")
    # 문장 전처리 수행
    sent = preprocessing(sent)
    # 전처리 후 문장이 비어있지 않으면 리스트에 추가
    if len(sent) > 0:
        train_sentences.append(sent)
        train_labels.append(label)

# 테스트 데이터 전처리
for i, (sent, label) in enumerate(zip(test_data['document'], test_data['label'])):
    # 1,000개마다 진행 상황 출력
    if i % 1000 == 0:
        print(f"Test processed = {i}")
    # 문장 전처리 수행
    sent = preprocessing(sent)
    # 전처리 후 문장이 비어있지 않으면 리스트에 추가
    if len(sent) > 0:
        test_sentences.append(sent)
        test_labels.append(label)

# 전처리에 걸린 총 시간 출력
print(time.time() - start)

Train processed = 0
Train processed = 10000
Train processed = 20000
Train processed = 30000
Train processed = 40000
Test processed = 0
Test processed = 1000
Test processed = 2000
Test processed = 3000
Test processed = 4000
126.25993275642395


In [8]:
# 훈련 레이블을 numpy 배열로 변환
train_labels = np.array(train_labels)

# 테스트 레이블을 numpy 배열로 변환
test_labels = np.array(test_labels)

print(train_labels.shape)
print(test_labels.shape)
print(train_labels)

(49571,)
(4951,)
[1 0 1 ... 1 0 0]


In [9]:
# 어휘 사전의 최대 크기 설정
VOCAB_SIZE = 20000

# Tokenizer 객체 생성 (최대 단어 수 지정 및 OOV(Out-Of-Vocabulary) 토큰 설정)
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")

# 훈련 문장에 대해 토크나이저 학습 수행 (단어 인덱스 구축)
tokenizer.fit_on_texts(train_sentences)

# 훈련 문장들을 시퀀스로 변환
train_sequences = tokenizer.texts_to_sequences(train_sentences)

# 테스트 문장들을 시퀀스로 변환
test_sequences = tokenizer.texts_to_sequences(test_sentences)

# 첫 번째 시퀀스 출력
print(train_sequences[0])
print(test_sequences[0])

[90, 1370, 13, 1554, 80, 520, 8870, 14637, 29, 250, 5, 3884, 16, 430, 62, 210, 30, 1612, 14, 744, 22, 229, 6, 1123, 13, 31, 43, 12, 149, 2547, 5, 741, 12, 1554, 14638, 6007, 8871, 8872, 8, 31]
[1683, 7, 460, 1491, 106, 346, 37, 2485, 344, 760, 206, 650, 96, 270]


In [10]:
max_length = 15

# 훈련 시퀀스를 패딩 처리 (최대 길이를 15로 설정, 'post' 방식으로 잘라내고 패딩)
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')

# 테스트 시퀀스를 패딩 처리 (최대 길이를 15로 설정, 'post' 방식으로 잘라내고 패딩)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post', truncating='post')

print(train_padded.shape)
print(test_padded.shape)

# 첫 번째 패딩된 시퀀스 출력
print(train_padded[0])
print(test_padded[0])

(49571, 15)
(4951, 15)
[   90  1370    13  1554    80   520  8870 14637    29   250     5  3884
    16   430    62]
[1683    7  460 1491  106  346   37 2485  344  760  206  650   96  270
    0]


In [ ]:
# 하이퍼파라미터 최적화를 통해 지정한 하이퍼파라미터들
# n-flod 방식이 아니기에, test 데이터에 과적합 되었을 수 있음
reservoir = Reservoir(200, sr=0.74, lr=0.9)  # 유닛 갯수 등 여러 하이퍼파라미터 지정
readout = Ridge(output_dim=1, ridge=0)  # 읽기 아웃 계층

reservoir <<= readout

# 모델 파이프라인 구성
esn = Input(input_dim=train_padded.shape[1]) >> reservoir >> readout

In [47]:
# 학습 파라미터 설정
epochs = 1 # 에포크 수

# 학습 과정 (배치 처리)
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    # 모델 학습
    # teachers forcing 사용
    esn=esn.fit(train_padded, train_labels.reshape(-1,1), force_teachers=True)

# train 데이터에 대한 평가
train_predictions_run = esn.run(train_padded)
predictions_int =[1 if y_p[0] >= 0.5 else 0 for y_p in train_predictions_run]  # 연속적인 출력을 0 또는 1로 이진화
accuracy = accuracy_score(train_labels, predictions_int)
print(f"정확도: {accuracy * 100:.2f}%")

Epoch 1/1


Running Model-5: 49571it [00:03, 12944.22it/s], ?it/s]
Running Model-5: 100%|██████████| 1/1 [00:03<00:00,  3.92s/it]


Fitting node Ridge-2...


Running Model-5: 49571it [00:03, 14783.99it/s]        

정확도: 53.13%


In [48]:
# 모델 테스트
# stateful=False를 통해 test 데이터로 상태를 변화시키지 않음
predictions = esn.run(test_padded, stateful=True)
predictions_int =[1 if y_p[0] >= 0.5 else 0 for y_p in predictions]  # 연속적인 출력을 0 또는 1로 이진화

# 성능 평가
accuracy = accuracy_score(test_labels, predictions_int)
print(f"정확도: {accuracy * 100:.2f}%")

Running Model-5: 4951it [00:00, 13351.73it/s]         

정확도: 51.77%


In [ ]:
# 테스트할 새로운 문장
new_sentence = "오랜만에 접한 수작"

new_sentence = preprocessing(new_sentence)
print(new_sentence)

# 새로운 문장을 토큰화
new_sequence = tokenizer.texts_to_sequences([new_sentence])
print(new_sequence)

# 패딩 적용
new_padded = pad_sequences(new_sequence, maxlen=max_length, padding='post', truncating='post')

# Reservoir 모델을 통해 상태 계산
predict = esn.run(new_padded, stateful=False)
print(predict)

# Ridge 모델을 통해 예측값 생성
prediction_int =1 if predict >= 0.5 else 0  # 연속적인 출력을 0 또는 1로 이진화

# 결과 출력
print(f"New Sentence: {new_sentence}")
print(f"Predicted Sentiment: {'Positive' if prediction_int == 1 else 'Negative'}")


['오랜', '만', '에', '접', '한', '수작']
[[349, 26, 7, 480, 22, 378]]


Running Model-7: 100%|██████████| 1/1 [00:00<?, ?it/s]

[[0.4791695]]
New Sentence: ['오랜', '만', '에', '접', '한', '수작']
Predicted Sentiment: Negative
